In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-classification-data/loan_data.csv


In [ ]:
"""
person_age	                    Age of the person	Float
person_gender	                Gender of the person	Categorical
person_education	            Highest education level	Categorical
person_income	                Annual income	Float
person_emp_exp	                Years of employment experience	Integer
person_home_ownership   	    Home ownership status (e.g., rent, own, mortgage)	Categorical
loan_amnt                   	Loan amount requested	Float
loan_intent             	    Purpose of the loan	Categorical
loan_int_rate	                Loan interest rate	Float
loan_percent_income     	    Loan amount as a percentage of annual income	Float
cb_person_cred_hist_length	    Length of credit history in years	Float
credit_score                	Credit score of the person	Integer
previous_loan_defaults_on_file	Indicator of previous loan defaults	Categorical
loan_status (target)	Loan approval status: 1 = approved; 0 = rejected	Integer

https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data
"""

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


data = pd.read_csv(r"/kaggle/input/loan-approval-classification-data/loan_data.csv")

In [3]:
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [4]:
data.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [5]:
categorical = ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']

In [ ]:
# Encoding the categorical data
data_1hot = pd.get_dummies(data, columns=categorical, drop_first=True, dtype= 'int32') 
data_1hot.head()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_male,...,person_education_Master,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_Yes
0,22.0,71948.0,0,35000.0,16.02,0.49,3.0,561,1,0,...,1,0,0,1,0,0,0,1,0,0
1,21.0,12282.0,0,1000.0,11.14,0.08,2.0,504,0,0,...,0,0,1,0,1,0,0,0,0,1
2,25.0,12438.0,3,5500.0,12.87,0.44,3.0,635,1,0,...,0,0,0,0,0,0,1,0,0,0
3,23.0,79753.0,0,35000.0,15.23,0.44,2.0,675,1,0,...,0,0,0,1,0,0,1,0,0,0
4,24.0,66135.0,1,35000.0,14.27,0.53,4.0,586,1,1,...,1,0,0,1,0,0,1,0,0,0


In [ ]:
#splitting
X = data_1hot.drop(['loan_status'], axis= 1)
Y = data_1hot['loan_status']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43, shuffle=True) 

In [ ]:
# Custom training function to train the data and evaluate it
def train(Model, X =X, Y=X,X_train = X_train, X_test = X_test, Y_train = Y_train, Y_test = Y_test):
    

    m = Model
    m.fit(X_train,Y_train)
    pred = m.predict(X_test)
    print('\t', Model)
    print(classification_report(Y_test,pred))



In [ ]:
#Training
train(DecisionTreeClassifier(max_depth=3, random_state=44),X,Y)
train(RandomForestClassifier(n_estimators=150, random_state=44),X,Y) 


	 DecisionTreeClassifier(max_depth=3, random_state=44)
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7042
           1       0.87      0.63      0.73      1958

    accuracy                           0.90      9000
   macro avg       0.89      0.80      0.84      9000
weighted avg       0.90      0.90      0.89      9000

	 RandomForestClassifier(n_estimators=150, random_state=44)
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      7042
           1       0.90      0.76      0.83      1958

    accuracy                           0.93      9000
   macro avg       0.92      0.87      0.89      9000
weighted avg       0.93      0.93      0.93      9000

